# Immobiliare Connector Test

This notebook demonstrates the usage of the immobiliare connector for scraping real estate data.

In [1]:
from pathlib import Path
import sys
import pandas as pd

# Find the project root (adjust as needed)
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path().cwd().parent

# Add sources to sys.path
sys.path.append(str(project_root / "sources"))
from connectors import ConfigManager
from connectors.immobiliare import ImmobiliareConnector, FileStorage

# If you need to pass a config path, do it like this:
config_path = "project_root /sources/connectors/immobiliare/config/default.yaml"

## Basic Usage

Let's start with a basic example of scraping a single page of real estate listings.

In [2]:
# Initialize config manager
config_manager = ConfigManager()

# Create connector with config
print("Creating connector...")
connector = ImmobiliareConnector(config_manager=config_manager)

# Example URL for Rome apartments
test_url = "https://www.immobiliare.it/vendita-case/milano/"

# Start scraping
print(f"\nStarting scraping from: {test_url}")
try:
    connector.scrape_and_store(test_url)
    print("Scraping completed successfully!")
except Exception as e:
    print(f"Error during scraping: {e}")


# Load and display the results
print("\nLoading results from CSV...")
try:
    # Get the storage instance from the connector
    storage = connector.storage
    df = pd.read_csv(storage.csv_file)
    
    # Display basic information
    print(f"\nTotal records: {len(df)}")
    print("\nColumns:")
    print(df.columns.tolist())
    
    # Display first few records
    print("\nFirst 5 records:")
    print(df.head())
    
    # Basic statistics
    print("\nPrice Statistics:")
    print(df['price'].describe())
    
    print("\nSurface Statistics:")
    print(df['surface'].describe())
    
    print("\nProperty Types:")
    print(df['property_type'].value_counts())
    
except Exception as e:
    print(f"Error analyzing results: {e}")

Creating connector...
Storage directory: C:\Users\gabri\workspace\aida_projects\quant-estate\.venv\Lib\data\immobiliare_data_2025-06-03
Storage initialized at: C:\Users\gabri\workspace\aida_projects\quant-estate\.venv\Lib\data\immobiliare_data_2025-06-03
Initialized CSV file: C:\Users\gabri\workspace\aida_projects\quant-estate\.venv\Lib\data\immobiliare_data_2025-06-03\immobiliare.csv

Starting scraping from: https://www.immobiliare.it/vendita-case/milano/


KeyboardInterrupt: 